# Modeling Notebook

## Summary
- In this Notebook I will be creating 3 simple models with:
    - Sklearn's Base Decision Tree
    - Catboost
    - Keras

I'll evaluate on all of these FSMs and decide how to move forward, and how to split my time between these model types.

# First Simple Models

In [17]:
# Import statements
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier, Pool, metrics, cv
import keras
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer

# Importing metrics function from functions.py
from functions import metrics as custom_score

In [18]:
# Load in cleaned data from last time.

# Training Data
X_train = pd.read_csv('../Data/train/X_train.csv', index_col=0)
y_train = pd.read_csv('../Data/train/y_train.csv', index_col=0)

# Testing Data
X_test = pd.read_csv('../Data/test/X_test.csv', index_col=0)
y_test = pd.read_csv('../Data/test/y_test.csv', index_col=0)


## Modeless Baseline
A modeless baseline is how accurate we would be if we guessed the majority class of our target variable. In this case, how accurate would we be if we guessed that the child in question did not have ADHD, for every child. This will be same for every split since we set the stratify parameter to true when we performed the TTS. It will be important to keep this metric in mind when modeling, to see when the model is just guessing the majority class.

In [39]:
y_train.value_counts(normalize=True)

K2Q31A
2.0       0.898907
1.0       0.101093
dtype: float64

In [14]:
# Getting % of each class and assigning to variables
no_adhd, adhd = y_train.value_counts(normalize=True)

# Printing modeless accuracy
print(f'If we said that each child in the set did not have ADHD, we would be {no_adhd*100:.0f}% accurate')


If we said that each child in the set did not have ADHD, we would be 90% accurate


## FSM - Sklearn Decision Tree

To start let's create the simplest model possible, to use as a baseline for future models. I'll use a Decision Tree model from Sklearn.

In [16]:
# Instantiating Tree
FSM_DT = DecisionTreeClassifier()

# Fitting Model
FSM_DT.fit(X_train, y_train)

# Score on the training data.
custom_score(y_train, FSM_DT.predict(X_train))

1.0

Wow! A perfect Score!
I guess we can go home, we did it, we solved ADHD

Just kidding, lets check the score on the testing data.

In [21]:
# Predictions
FSM_DT_preds = FSM_DT.predict(X_test)

# Print metrics
custom_score(y_test, FSM_DT_preds)

Accuracy: 0.90
Precision: 0.49
Recall: 0.54
F1: 0.51
ROC AUC: 0.74


In [25]:
# Checking the predictions to ensure that the model isn't guessing one class
pd.Series(FSM_DT_preds).value_counts()

2.0    7104
1.0     882
dtype: int64

## Analysis
The model is, obviously, overfit to the training data, and on the testing data it may as well be guessing. This is expected of an un-pruned DT model though.
I'd like to try a few different kinds of first models, so before iterating on this one let's create a few more FSMs.

## FSM - CatBoost
Catboost is not an Sklearn library, but is known for doing very well on categorical data like the one from this survey. Let's give it a shot and see how it does.

In [30]:
# Setting up the model
model = CatBoostClassifier(
    # Adding Accuracy as a metric
    custom_loss=[metrics.Accuracy()],
    random_seed=15,
    logging_level='Silent'
)

In [31]:
# Fitting the model to training data
model.fit(
    X_train, y_train,
    # Using X/y test as eval set
    eval_set=(X_test, y_test),
    # Plot the learning of the model
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [43]:
# Updating model params with Logloss function
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
# Pooling data and cross validating
cv_data = cv(
    Pool(X_train, y_train),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [44]:
# Printing training and testing scores.
print("Training Scores")
custom_score(y_train, model.predict(X_train))
print('\t')
print('Testing Scores')
custom_score(y_test, model.predict(X_test))

Training Scores
Accuracy: 0.97
Precision: 0.93
Recall: 0.73
F1: 0.82
ROC AUC: 0.86
	
Testing Scores
Accuracy: 0.94
Precision: 0.74
Recall: 0.56
F1: 0.64
ROC AUC: 0.77


### Analysis
Catboost has done quite well for a baseline model! There is, again, some bad overfitting occuring here. But the scores are still better then the base decision tree; precision, recall, and roc/auc have all vastly improved compared to the base decision tree. Lets move on to something totally different, a neural network

## FSM - Keras NN

In [36]:
# Currently the classes are labelled as "1" as dignosed ADHD and "2" as not diagnosed, but Keras seems to dislike this.
lb = LabelBinarizer()
# Using sklearn LabelBinarizer to change classes to "0" and "1" so that Keras doesn't get confused.
labels = lb.fit_transform(y_train)
test_labels = lb.transform(y_test)

In [31]:
# Instantiating a NN
FSM_NN = keras.Sequential()

# Starting small with 30 neurons
FSM_NN.add(Dense(30, 'relu', input_shape=(422,)))

# 1 output
FSM_NN.add(Dense(1, 'sigmoid'))

# Compiling model with accuracy, precision, and recall metrics. Using "Adam" as an optimizer
FSM_NN.compile('adam', 'binary_crossentropy', metrics=['acc', 'Precision', 'Recall'])

In [38]:
# Fitting model on X_train and binarized labels
FSM_NN.fit(X_train, labels, epochs=10, steps_per_epoch=100, validation_data=(X_test, test_labels))

Epoch 1/10
100/100 [==============================] - 1s 12ms/step - loss: 5.2351 - acc: 0.8992 - precision: 0.9440 - recall: 0.9438 - val_loss: 5.2315 - val_acc: 0.9154 - val_precision: 0.9282 - val_recall: 0.9817
Epoch 2/10
100/100 [==============================] - 1s 11ms/step - loss: 4.6258 - acc: 0.8979 - precision: 0.9441 - recall: 0.9422 - val_loss: 44.9626 - val_acc: 0.8988 - val_precision: 0.8988 - val_recall: 1.0000
Epoch 3/10
100/100 [==============================] - 1s 8ms/step - loss: 9.7854 - acc: 0.8925 - precision: 0.9389 - recall: 0.9418 - val_loss: 8.9120 - val_acc: 0.9162 - val_precision: 0.9313 - val_recall: 0.9790
Epoch 4/10
100/100 [==============================] - 1s 10ms/step - loss: 8.3239 - acc: 0.8954 - precision: 0.9422 - recall: 0.9413 - val_loss: 8.3513 - val_acc: 0.9164 - val_precision: 0.9397 - val_recall: 0.9691
Epoch 5/10
100/100 [==============================] - 1s 9ms/step - loss: 4.0571 - acc: 0.9133 - precision: 0.9512 - recall: 0.9525 - val_lo

In [43]:
# Getting stats for test data
NN_loss, NN_acc, NN_prec, NN_recall = FSM_NN.evaluate(X_test, test_labels)

250/250 [==============================] - 1s 2ms/step - loss: 3.9140 - acc: 0.8847 - precision: 0.9583 - recall: 0.9114


In [46]:
# Neatly printing evaluation results
print(f'Test Accuracy: {NN_acc:.2f} \n Test Precision: {NN_prec:.2f} \n Test Recall: {NN_recall:.2f}')

Test Accuracy: 0.88 
 Test Precision: 0.96 
 Test Recall: 0.91


## Analysis
The neural network does suprisingly well with very few layers and neurons, with some additional layers/tweaking I'm sure we could improve the score. The accuracy is currently below the modeless baseline, and the precision and recall scores indicate that it isn't just choosing one class.

# Conclusion
Both Catboost and the base Decision tree did well in accuracy, but had poor recall/precision scores. The neural network a little worse accuracy wise, but had very high precision and recall scores. Moving forward this is what I'm planning:

1. Spend a very small amount of time on the base decision tree, perhaps iterate only once or twice.

2. Catboost will be the way to go here, I'll spend the most time iterating off of this model.

3. Spend a moderate amount of time on the neural network, and see if it will do better then Catboost.

I think that Catboost has the most potential here, and I'm confident that it will do better then a base decision tree ever could. I think it will be interesting to see what happens with the Neural network. It has good scores, but I'm wondering if it will be able to do much better then Catboost, and if it can, will it be worth the tradeoff of training time and processing power?